In [ ]:
from pyincore import IncoreClient
from pyincore.analyses.capitalshocks import CapitalShocks
from pyincore.analyses.seasidecge import SeasideCGEModel

In [ ]:
from pyincore import Dataset, FragilityService, MappingSet, DataService
from pyincore.analyses.buildingdamage import BuildingDamage
from pyincore.analyses.montecarlofailureprobability import MonteCarloFailureProbability

import pandas as pd


In [ ]:
client = IncoreClient()
data_service = DataService(client)

First step: run capital shocks analysis: It needs the building to sector mapping table.
Capital stock shocks for an individual building is equal to the functionality probability multiplied by value of the building.

Modeling residential properties is straightforward because we have the addresses for each property and can group homes into different valuations and occupancy structures (i.e. HS1 ≤ $100,000, HS2 > $100,000, HS3 – rentals).

In [ ]:
# Earthquake MCS Analysis: 
    
hazard_type = "earthquake"
rt = [1000]
rt_hazard_dict = {1000: "5dfa3e36b9219c934b64c231"}

In [ ]:
# Reading the updated building inventory: It contains the strctid
# Temporary Seaside, OR Building Inventory v6
building_inv = Dataset.from_file("IN-CORE_Seaside_BuildingInventory_2021-03-19.shp", data_type='ergo:buildingInventoryVer6')

bldg_dataset_id = "613ba5ef5d3b1d6461e8c415"        # defining building dataset (GIS point layer)
building_inv = Dataset.from_data_service(bldg_dataset_id, data_service)


In [ ]:
# Calculate the building damage due to earthquake:
from pyincore.analyses.buildingdamage import BuildingDamage

bldg_eq_dmg_result_list = []  # place holder to saving earthquake building damage result iteration
#bldg_dataset_id = "5df40388b9219c06cf8b0c80"        # defining building dataset (GIS point layer);
mapping_id = "5d2789dbb9219c3c553c7977"             # specifiying mapping id from fragilites to building types
fragility_service = FragilityService(client)        # loading fragility mapping
mapping_set = MappingSet(fragility_service.get_mapping(mapping_id))

for rt_val in rt:                                       # loop through recurrence interval
    bldg_dmg = BuildingDamage(client)                   # initializing pyincore
    #bldg_dmg.load_remote_input_dataset("buildings", bldg_dataset_id) # loading in the above id
    bldg_dmg.set_input_dataset("buildings", building_inv)    # loading the shapefile directly! 
    mapping_set = MappingSet(fragility_service.get_mapping(mapping_id))
    bldg_dmg.set_input_dataset("dfr3_mapping_set", mapping_set)

    result_name = 'buildings_eq_' + str(rt_val) + 'yr_dmg_result' # defining output name

    bldg_dmg.set_parameter("hazard_type", hazard_type)  # defining hazard type (e.g. earthquake vs. tsunami)
    hazard_id = rt_hazard_dict[rt_val]                  # specifying hazard id for specific recurrence interval
    bldg_dmg.set_parameter("hazard_id", hazard_id)      # loading above into pyincore
    bldg_dmg.set_parameter("fragility_key", "Non-Retrofit Fragility ID Code")
    bldg_dmg.set_parameter("num_cpu", 6)                # number of CPUs to use for parallel processing
    bldg_dmg.set_parameter("result_name", result_name)  # specifying output name in pyincore

    bldg_dmg.run_analysis()                             # running the analysis with the above parameters: it gives you 
    bldg_eq_dmg_result_list.append(bldg_dmg.get_output_dataset('ds_result'))

In [ ]:
#Monte Carlo Building Failure

mc_bldg_eq_dmg_result_list = []        # place holder to saving mc earthquake building damage result iteration
mc = MonteCarloFailureProbability(client)     # initializing pyincore

for index, rt_val in enumerate(rt):
    print('Recurrence Interval: {}' .format(rt_val))
    # --- earthquake MC failure ---
    result_name = 'mc_buildings_eq_' + str(rt_val) + 'yr'

    # running mc sampling in pyincore
    damage_dataset = bldg_eq_dmg_result_list[index]
    mc.set_input_dataset("damage", damage_dataset)           # importing csv to pyincore
    mc.set_parameter("result_name", result_name)             # defining result_name
    mc.set_parameter("num_cpu", 1)                           # number of CPUs for parallel processing
    mc.set_parameter("num_samples", 1000)                     # number of MC samples
    mc.set_parameter("damage_interval_keys", ["DS_0", "DS_1", "DS_2", "DS_3"])  # damage interval keys
    mc.set_parameter("failure_state_keys", ["DS_1", "DS_2", "DS_3"])   # damage keys classified as "failure"
    # water and power effects!
    mc.run_analysis()                                        # running MC analysis
    mc_bldg_eq_dmg_result_list.append(mc.get_output_dataset('failure_probability'))




In [ ]:
mc_bldg_eq_dmg_result_list

In [ ]:
building_failure_probability = mc.get_output_dataset('failure_probability')  # get buildings failure probabilities

df_bldg_fail = building_failure_probability.get_dataframe_from_csv()
df_bldg_fail.head()

In [ ]:
# Capital Shock Analysis
# shock is defined as of percentage reductions in the capital stock of the associated sectors.  
capital_shocks = CapitalShocks(client)

# Specify the result name
result_name = "Seaside capital losses"

# Set analysis parameters
capital_shocks.set_parameter("result_name", result_name)


# Load input datasets
# Seaside building to sector mapping table
building_to_sectors_id = "5f5fd8e2980a301080a03996"


# Load remote datasets
capital_shocks.load_remote_input_dataset("buildings_to_sectors", building_to_sectors_id)

# Set datasets
# Seaside building inventory
capital_shocks.set_input_dataset("buildings", building_inv)  # loading the shapefile directly! 



In [ ]:
# Seaside building failure probability
capital_shocks.set_input_dataset("failure_probability", building_failure_probability)
capital_shocks.run_analysis()


In [ ]:
sector_shocks_result = capital_shocks.get_output_dataset("sector_shocks")
df = sector_shocks_result.get_dataframe_from_csv()
df

In [ ]:
# CGE model: 

# SAM
# Social accounting matrix (SAM) contains data for firms, households and government which are organized 
# in a way to represent the interactions of all three entities in a typical economy
## each cell shows payment from its column account to its row account
SAM = "5f17393633b2700c11feab38"

# CAPITAL COMP
# BB is a matrix which describes how investment in physical infrastructure is transformed into functioning capital such as commercial and residential buildings. 
# These data are collected from the Bureau of Economic Analysis (BEA).
BB = "603d23d634f29a7fa41c4c4e"

# MISC TABLES
EMPLOY = "5f173c97c98cf43417c8955d" # Table name containing data for commercial sector employment.
JOBCR = "5f173d262fab4d660a8e9f9c" # This is a matrix describing the supply of workers coming from each household group in the economy.
HHTABLE = "5f173d6bc98cf43417c89561" # HH Table?
SIMS = "5f174211c98cf43417c89565" # Random numbers for the change of capital stocks in the CGE model.

# The reference to this object id is no longer needed as we create the sector shocks object above
#sector_shocks = "5f5fdbc1b7745919a9e2482d" # This is the aggregation of building functionality states to capital shocks per sector; 100yr earthquake damage

In [ ]:
# Create Seaside CGE Model
seaside_cge = SeasideCGEModel(client)

In [ ]:
# Set analysis input datasets
seaside_cge.load_remote_input_dataset("SAM", SAM)
seaside_cge.load_remote_input_dataset("BB", BB)
seaside_cge.load_remote_input_dataset("EMPLOY", EMPLOY)
seaside_cge.load_remote_input_dataset("JOBCR", JOBCR)
seaside_cge.load_remote_input_dataset("HHTABLE", HHTABLE)
seaside_cge.load_remote_input_dataset("SIMS", SIMS)
#seaside_cge.load_remote_input_dataset("sector_shocks", sector_shocks)

# Set analysis input dataset from previous Capital stock shock analysis
seaside_cge.set_input_dataset("sector_shocks", sector_shocks_result)

Troubleshooting Solver Path Issues (only if encountered)
If your code can't run and gives you error, "Invalid executable path, please make sure you have Pyomo installed.", Then, you don't have "ipopt" properly installed.

The solver_path variable points to the absolute path for where the ipopt solver is installed. To know the absolute path of the ipopt solver in your local machine, open a terminal window, activate your pyIncore environment, and run the following command according to your OS:

on Mac: open a terminal window, activate your conda environment and type "which ipopt". Paste the resulting path below.
on Windows: open a command terminal, activate your conda environment and type "where ipopt". Paste the resulting path below.

In [ ]:
# Set analysis parameters (optional)
#seaside_cge.set_parameter("solver_path", "ipopt")
#seaside_cge.set_parameter("print_solver_output", False)

In [ ]:
# Run Seaside CGE model analysis
seaside_cge.run_analysis()

In [ ]:
# A dataset containing Seaside cge simulations (format: CSV).
# dsc+dsr = domestic supply
domestic_supply = seaside_cge.get_output_dataset('Seaside_Sims')
df_fp = domestic_supply.get_dataframe_from_csv()
df_fp


In [ ]:
# A dataset of changes in employment and supply. (format: CSV).
# Employment (unit = person)
gross_income = seaside_cge.get_output_dataset('Seaside_output')
df_fpp = gross_income.get_dataframe_from_csv()
df_fpp